# DA Practice 1: Data Science Salaries


[Data modeling](https://en.wikipedia.org/wiki/Data_modeling) is a crucial process in information management that involves the creation of a conceptual, logical, and physical representation of data. It provides a structured approach for organizing and storing data in a database, enabling efficient access and retrieval of information. Effective data modeling ensures the accuracy, consistency, and integrity of data by establishing clear relationships between data elements and defining their attributes.

In this report, I will provide an overview of data modeling, including its importance, process, and techniques used in developing a data model. I will then demonstrate the application of data modeling by using a dataset that the teacher sent from kaggle. Additionally, I will explain the steps involved in the data modeling process. This is to cover modeling aspects that I can still imporve on.

## Problem definition

**Research question: How do we build a model that is able to predict the salary?**

The main focus of this project is to build a model that is able to predict the salary, this is done using a few feature columns related to salary (salary, salary in USD, salary currency)

Other questions to answer:

How should you deal with data cleaning? 
How to deal with the job titles?
How to deal with the many countries?
Which is the best hyper-parameters for the decision tree?